In [ ]:
import os


os.chdir("../")


os.getcwd()


'd:\\DA_workspace\\car_price_prediction\\src'

In [ ]:
import pandas as pd
import numpy as np
from regressor.Mylib import myfuncs
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split


Doc du lieu


In [ ]:
df = myfuncs.load_python_object("../artifacts/data_ingestion/train_data.pkl")

df.head()


,Brand,Model,Year,Engine_Size,Fuel_Type,Transmission,Mileage,Doors,Owner_Count,Price
0,Toyota,Camry,2004,4.4,Diesel,Semi-Automatic,137461,4,5,6950
1,Honda,CR-V,2015,4.3,Electric,Semi-Automatic,109966,2,1,12700
2,Chevrolet,Impala,2006,3.0,Electric,Manual,9880,4,5,10702
3,Toyota,RAV4,2023,2.4,Petrol,Semi-Automatic,240972,2,3,8580
4,Volkswagen,Passat,2017,2.7,Diesel,Automatic,167257,4,2,10054


Kich thuoc


In [ ]:
df.shape


(9000, 10)

Ti le missing cac cot


In [ ]:
df.isnull().mean() * 100


Brand           0.0
Model           0.0
Year            0.0
Engine_Size     0.0
Fuel_Type       0.0
Transmission    0.0
Mileage         0.0
Doors           0.0
Owner_Count     0.0
Price           0.0
dtype: float64

Khong xoa cot nao het !


Kich thuoc con lai


In [ ]:
df.shape


(9000, 10)

Y nghia cac cot

| Cot          | Y nghia                 | Don vi | Phan loai |
| ------------ | ----------------------- | ------ | --------- |
| Brand        | Nhan hieu               | none   | Nominal   |
| Model        | Mo hinh xe              | none   | Nominal   |
| Year         | Nam san xuat xe         | none   | Numeric   |
| Engine_Size  | kich thuoc dong co      | lit    | Numeric   |
| Fuel_Type    | Loai nhien lieu su dung | none   | Nominal   |
| Transmission | Loai hop so             | none   | Nominal   |
| Mileage      | Quang duong xe da di    | km     | Numeric   |
| Doors        | So cua xe               | none   | Numeric   |
| Owner_Count  | So nguoi chu cu cua xe  | none   | Numeric   |
| Price        | Gia ban                 | none   | Numeric   |


Doi ten cot


In [ ]:
df.columns


Index(['Brand', 'Model', 'Year', 'Engine_Size', 'Fuel_Type', 'Transmission',
       'Mileage', 'Doors', 'Owner_Count', 'Price'],
      dtype='object')

In [ ]:
rename_dict = {
    "Brand": "Brand_nom",
    "Model": "Model_nom",
    "Year": "Year_num",
    "Engine_Size": "Engine_Size_num",
    "Fuel_Type": "Fuel_Type_nom",
    "Transmission": "Transmission_type_nom",
    "Mileage": "Mileage_num",
    "Doors": "Doors_count_num",
    "Owner_Count": "old_Owner_Count_num",
}


df = df.rename(columns=rename_dict)

df.head()


,Brand_nom,Model_nom,Year_num,Engine_Size_num,Fuel_Type_nom,Transmission_type_nom,Mileage_num,Doors_count_num,old_Owner_Count_num,Price
0,Toyota,Camry,2004,4.4,Diesel,Semi-Automatic,137461,4,5,6950
1,Honda,CR-V,2015,4.3,Electric,Semi-Automatic,109966,2,1,12700
2,Chevrolet,Impala,2006,3.0,Electric,Manual,9880,4,5,10702
3,Toyota,RAV4,2023,2.4,Petrol,Semi-Automatic,240972,2,3,8580
4,Volkswagen,Passat,2017,2.7,Diesel,Automatic,167257,4,2,10054


In [ ]:
df.columns


Index(['Brand_nom', 'Model_nom', 'Year_num', 'Engine_Size_num',
       'Fuel_Type_nom', 'Transmission_type_nom', 'Mileage_num',
       'Doors_count_num', 'old_Owner_Count_num', 'Price'],
      dtype='object')

Y nghia cac cot

| Cot                   | Y nghia                 | Don vi | Phan loai |
| --------------------- | ----------------------- | ------ | --------- |
| Brand_nom             | Nhan hieu               | none   | Nominal   |
| Model_nom             | Mo hinh xe              | none   | Nominal   |
| Year_num              | Nam san xuat xe         | none   | Numeric   |
| Engine_Size_num       | kich thuoc dong co      | lit    | Numeric   |
| Fuel_Type_nom         | Loai nhien lieu su dung | none   | Nominal   |
| Transmission_type_nom | Loai hop so             | none   | Nominal   |
| Mileage_num           | Quang duong xe da di    | km     | Numeric   |
| Doors_count_num       | So cua xe               | none   | Numeric   |
| old_Owner_Count_num   | So nguoi chu cu cua xe  | none   | Numeric   |
| Price                 | Gia ban                 | none   | Numeric   |


Chuyen cac cot numeric, binary, nominal, ordinal, target ve dung vi tri


In [ ]:
df.columns


Index(['Brand_nom', 'Model_nom', 'Year_num', 'Engine_Size_num',
       'Fuel_Type_nom', 'Transmission_type_nom', 'Mileage_num',
       'Doors_count_num', 'old_Owner_Count_num', 'Price'],
      dtype='object')

In [ ]:
cols = pd.Series(df.columns)
numeric_cols = cols[cols.str.endswith("num")].tolist()
binary_cols = cols[cols.str.endswith("bin")].tolist()
nominal_cols = cols[cols.str.endswith("nom")].tolist()
ordinal_cols = cols[cols.str.endswith("ord")].tolist()


target_col = "Price"


df = df[numeric_cols + binary_cols + nominal_cols + ordinal_cols + [target_col]]


df.head()


,Year_num,Engine_Size_num,Mileage_num,Doors_count_num,old_Owner_Count_num,Brand_nom,Model_nom,Fuel_Type_nom,Transmission_type_nom,Price
0,2004,4.4,137461,4,5,Toyota,Camry,Diesel,Semi-Automatic,6950
1,2015,4.3,109966,2,1,Honda,CR-V,Electric,Semi-Automatic,12700
2,2006,3.0,9880,4,5,Chevrolet,Impala,Electric,Manual,10702
3,2023,2.4,240972,2,3,Toyota,RAV4,Petrol,Semi-Automatic,8580
4,2017,2.7,167257,4,2,Volkswagen,Passat,Diesel,Automatic,10054


Kiem tra kieu du lieu cac cot


In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Year_num               9000 non-null   int64  
 1   Engine_Size_num        9000 non-null   float64
 2   Mileage_num            9000 non-null   int64  
 3   Doors_count_num        9000 non-null   int64  
 4   old_Owner_Count_num    9000 non-null   int64  
 5   Brand_nom              9000 non-null   object 
 6   Model_nom              9000 non-null   object 
 7   Fuel_Type_nom          9000 non-null   object 
 8   Transmission_type_nom  9000 non-null   object 
 9   Price                  9000 non-null   int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 703.2+ KB


DUNG

- Year_num
- Engine_Size_num
- Mileage_num
- Doors_count_num
- old_Owner_Count_num
- Price

---

SAI

- Brand_nom
- Model_nom
- Fuel_Type_nom
- Transmission_type_nom


Kiem tra cac cot co kdl = kdlmm + NA


In [ ]:
set(map(type, df["Brand_nom"]))


{str}

In [ ]:
set(map(type, df["Model_nom"]))


{str}

In [ ]:
set(map(type, df["Fuel_Type_nom"]))


{str}

In [ ]:
set(map(type, df["Transmission_type_nom"]))


{str}

Kiem tra noi dung cac cot

Cot numeric


In [ ]:
df[numeric_cols].describe().loc[["min", "max", "mean"]]


,Year_num,Engine_Size_num,Mileage_num,Doors_count_num,old_Owner_Count_num
min,2000.000000,1.000000,25.000,2.000000,1.000000
max,2023.000000,5.000000,299947.000,5.000000,5.000000
mean,2011.539222,2.999789,149348.213,3.501444,2.991778


Cot binary


In [ ]:
for col in binary_cols:
    print(f"{col} -> {df[col].unique().tolist()}")


Cot nominal


In [ ]:
for col in nominal_cols:
    print(f"{col} -> {df[col].unique().tolist()}")


Brand_nom -> ['Toyota', 'Honda', 'Chevrolet', 'Volkswagen', 'Ford', 'Kia', 'Hyundai', 'BMW', 'Mercedes', 'Audi']
Model_nom -> ['Camry', 'CR-V', 'Impala', 'RAV4', 'Passat', 'Fiesta', 'Sportage', 'Elantra', '5 Series', '3 Series', 'Equinox', 'Focus', 'Accord', 'E-Class', 'Rio', 'Explorer', 'Sonata', 'Tucson', 'Optima', 'X5', 'A3', 'A4', 'C-Class', 'Q5', 'Malibu', 'GLA', 'Corolla', 'Tiguan', 'Golf', 'Civic']
Fuel_Type_nom -> ['Diesel', 'Electric', 'Petrol', 'Hybrid']
Transmission_type_nom -> ['Semi-Automatic', 'Manual', 'Automatic']


Cot ordinal


In [ ]:
for col in ordinal_cols:
    print(f"{col} -> {df[col].unique().tolist()}")


Cot target


In [ ]:
print(f"{target_col} -> {df[target_col].unique().tolist()}")


Price -> [6950, 12700, 10702, 8580, 10054, 7091, 13490, 6827, 8839, 7642, 11848, 11671, 12488, 9831, 8547, 10710, 4970, 9075, 12909, 9723, 12509, 9986, 3554, 5947, 5977, 14202, 8366, 13875, 11316, 8990, 8541, 11372, 13650, 9355, 14519, 3126, 10459, 9643, 5782, 10340, 10186, 7277, 10866, 7275, 11474, 8654, 13849, 12296, 8222, 9704, 11534, 10215, 12972, 4688, 11478, 12447, 2000, 12282, 10121, 5651, 12394, 12056, 8842, 6806, 5328, 10267, 12112, 10423, 10745, 5863, 7456, 8091, 6858, 13663, 10962, 12929, 6868, 8878, 6904, 4659, 5811, 7010, 13435, 11322, 13588, 9839, 10354, 7876, 11357, 7533, 7458, 10623, 8124, 6461, 6724, 8457, 10830, 12020, 9261, 10066, 13962, 10000, 6456, 5461, 13104, 11337, 9612, 6453, 8327, 5503, 6896, 5286, 7491, 12650, 6485, 9693, 7803, 10045, 12616, 2903, 11127, 9412, 6208, 10686, 11799, 9225, 13743, 9435, 7568, 4545, 9618, 4524, 11559, 12777, 6726, 9148, 5594, 5938, 12560, 7060, 9221, 8854, 10609, 7762, 12587, 11759, 2478, 6625, 4759, 8734, 3115, 7911, 6854, 9639, 1

Fill missing value


In [ ]:
cat_cols = binary_cols + nominal_cols + ordinal_cols + [target_col]

preprocessor = ColumnTransformer(
    transformers=[
        ("num", SimpleImputer(strategy="mean"), numeric_cols),
        ("cat", SimpleImputer(strategy="most_frequent"), cat_cols),
    ]
)

df_filled = preprocessor.fit_transform(df)
df_filled = pd.DataFrame(df_filled, columns=numeric_cols + cat_cols)

# Luu preprocessor lai
folder = "../artifacts/data_correction"
os.makedirs(folder, exist_ok=True)
myfuncs.save_python_object(
    file_path=os.path.join(folder, "missing_value_handler.pkl"), obj=preprocessor
)

df_filled.head()


,Year_num,Engine_Size_num,Mileage_num,Doors_count_num,old_Owner_Count_num,Brand_nom,Model_nom,Fuel_Type_nom,Transmission_type_nom,Price
0,2004.0,4.4,137461.0,4.0,5.0,Toyota,Camry,Diesel,Semi-Automatic,6950
1,2015.0,4.3,109966.0,2.0,1.0,Honda,CR-V,Electric,Semi-Automatic,12700
2,2006.0,3.0,9880.0,4.0,5.0,Chevrolet,Impala,Electric,Manual,10702
3,2023.0,2.4,240972.0,2.0,3.0,Toyota,RAV4,Petrol,Semi-Automatic,8580
4,2017.0,2.7,167257.0,4.0,2.0,Volkswagen,Passat,Diesel,Automatic,10054


Chuyen doi ve dung kieu du lieu


Kiem tra cac gia tri cua cot binary


In [ ]:
for col in binary_cols:
    print(f"{col} -> {df[col].unique().tolist()}")


Chuyen cac cot binary ve so


In [ ]:
df_filled[binary_cols] = df_filled[binary_cols].replace(
    {
        "Yes": 1,
        "No": 0,
    }
)


In [ ]:
df_filled[numeric_cols + binary_cols] = df_filled[numeric_cols + binary_cols].astype(
    "float"
)


df_filled[nominal_cols + ordinal_cols] = df_filled[nominal_cols + ordinal_cols].astype(
    "category"
)

df_filled[target_col] = df_filled[target_col].astype("float")

df_filled.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   Year_num               9000 non-null   float64 
 1   Engine_Size_num        9000 non-null   float64 
 2   Mileage_num            9000 non-null   float64 
 3   Doors_count_num        9000 non-null   float64 
 4   old_Owner_Count_num    9000 non-null   float64 
 5   Brand_nom              9000 non-null   category
 6   Model_nom              9000 non-null   category
 7   Fuel_Type_nom          9000 non-null   category
 8   Transmission_type_nom  9000 non-null   category
 9   Price                  9000 non-null   float64 
dtypes: category(4), float64(6)
memory usage: 459.1 KB


Thay doi thu tu cac label cho cac cot ordinal


In [ ]:
ord_values_dict = {
    "Physical_Activity_Level_ord": ["Low", "Medium", "High"],
    "Smoking_Status_level_ord": ["Never", "Former", "Current"],
    "Alcohol_Consumption_Level_ord": ["Never", "Occasionally", "Regularly"],
    "Cholesterol_Level_ord": ["Normal", "High"],
    "Depression_Level_ord": ["Low", "Medium", "High"],
    "Sleep_Quality_Level_ord": ["Poor", "Average", "Good"],
    "Diet_level_ord": ["Unhealthy", "Average", "Healthy"],
    "Air_Pollution_Exposure_level_ord": ["Low", "Medium", "High"],
    "Social_Engagement_Level_ord": ["Low", "Medium", "High"],
    "Income_Level_ord": ["Low", "Medium", "High"],
    "Stress_Levels_ord": ["Low", "Medium", "High"],
}

ordinal_cols_series = pd.Series(df_filled.columns)
ordinal_cols_series = ordinal_cols_series[
    ordinal_cols_series.str.endswith("ord")
].tolist()


for col in ordinal_cols_series:
    df_filled[col] = df_filled[col].cat.reorder_categories(
        ord_values_dict[col], ordered=True
    )


Check duplicates


In [ ]:
any(df_filled.duplicated())


False

Khong co duplicates


In [ ]:
df_filled.shape


(9000, 10)

Chia thanh tap train va tap val


In [ ]:
df_train, df_val = train_test_split(df_filled, test_size=0.1, random_state=42)

myfuncs.save_python_object(os.path.join(folder, "train_data.pkl"), df_train)
myfuncs.save_python_object(os.path.join(folder, "val_data.pkl"), df_val)
